In [1]:
import numpy as np
import pandas
import pm4py
from matplotlib import pyplot as plt
from sklearn.mixture import GaussianMixture
import scipy.stats as stats
import ot
import os
from tqdm import tqdm
import collections
import matplotlib.dates as md
import importlib
import pickle
import random
import math
import CRPS.CRPS as pscore
import datetime

pandas.set_option('display.max_columns', None)
#pandas.set_option('display.max_rows', None)


import sys
sys.path.append('../../TaskExecutionTimeMining/')
from drbart_parser import *
from event_log_transformer import *

#sys.path.append('../../Evaluation')
sys.path.append('../../Evaluation/')
from normal_evaluation.drbart_evaluation import *

In [2]:
#model_name = 'bpic_2017_all_2'
model_name = 'bpic_2017_all'

log_name = 'test'
with open('../transformed_event_logs/BPIC_2017_all_'+log_name+'.pickle', 'rb') as f:
    test_event_log = pickle.load(f)

test_event_log['case:concept:name'] = test_event_log['case:concept:name'].astype(str)
known_resources = ['User_1','User_10','User_100','User_101','User_102','User_103','User_104','User_105','User_106','User_107','User_108','User_109','User_11','User_110','User_111','User_112','User_113','User_114','User_115','User_116','User_117','User_118','User_119','User_12','User_120','User_121','User_122','User_123','User_124','User_125','User_126','User_127','User_128','User_129','User_13','User_130','User_131','User_132','User_133','User_134','User_135','User_136','User_137','User_138','User_139','User_14','User_140','User_141','User_142','User_143','User_144','User_145','User_146','User_147','User_148','User_149','User_15','User_16','User_17','User_18','User_19','User_2','User_20','User_21','User_22','User_23','User_24','User_25','User_26','User_27','User_28','User_29','User_3','User_30','User_31','User_32','User_33','User_34','User_35','User_36','User_37','User_38','User_39','User_4','User_40','User_41','User_42','User_43','User_44','User_45','User_46','User_47','User_48','User_49','User_5','User_50','User_51','User_52','User_53','User_54','User_55','User_56','User_57','User_58','User_59','User_6','User_60','User_61','User_62','User_63','User_64','User_65','User_66','User_67','User_68','User_69','User_7','User_70','User_71','User_72','User_73','User_74','User_75','User_76','User_77','User_78','User_79','User_8','User_80','User_81','User_82','User_83','User_84','User_85','User_86','User_87','User_88','User_89','User_9','User_90','User_91','User_92','User_93','User_94','User_95','User_96','User_97','User_98','User_99']
known_activities = ['W_Assess potential fraud__ate_abort','W_Assess potential fraud__complete','W_Assess potential fraud__resume','W_Assess potential fraud__schedule','W_Assess potential fraud__start','W_Assess potential fraud__suspend','W_Assess potential fraud__withdraw','W_Call after offers__ate_abort','W_Call after offers__complete','W_Call after offers__resume','W_Call after offers__schedule','W_Call after offers__start','W_Call after offers__suspend','W_Call after offers__withdraw','W_Call incomplete files__ate_abort','W_Call incomplete files__complete','W_Call incomplete files__resume','W_Call incomplete files__schedule','W_Call incomplete files__start','W_Call incomplete files__suspend','W_Complete application__ate_abort','W_Complete application__complete','W_Complete application__resume','W_Complete application__schedule','W_Complete application__start','W_Complete application__suspend','W_Handle leads__complete','W_Handle leads__resume','W_Handle leads__schedule','W_Handle leads__start','W_Handle leads__suspend','W_Handle leads__withdraw','W_Shortened completion __resume','W_Shortened completion __schedule','W_Shortened completion __start','W_Shortened completion __suspend','W_Validate application__ate_abort','W_Validate application__complete','W_Validate application__resume','W_Validate application__schedule','W_Validate application__start','W_Validate application__suspend']

/tmp/ipykernel_2111361/2121463303.py:6: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  test_event_log = pickle.load(f)


In [3]:
test_event_log

,Action_start,org:resource_start,concept:name,EventOrigin_start,EventID_start,lifecycle:transition_start,time:timestamp_start,case:LoanGoal_start,case:ApplicationType_start,case:concept:name,case:RequestedAmount_start,FirstWithdrawalAmount_start,NumberOfTerms_start,Accepted_start,MonthlyCost_start,Selected_start,CreditScore_start,OfferedAmount_start,OfferID_start,Action_complete,org:resource_complete,EventOrigin_complete,EventID_complete,lifecycle:transition_complete,time:timestamp_complete,case:LoanGoal_complete,case:ApplicationType_complete,case:RequestedAmount_complete,FirstWithdrawalAmount_complete,NumberOfTerms_complete,Accepted_complete,MonthlyCost_complete,Selected_complete,CreditScore_complete,OfferedAmount_complete,OfferID_complete,duration,duration_seconds,duration_ms,duration_hours,seconds_in_day,day_of_week,User_1,User_10,User_100,User_101,User_102,User_103,User_104,User_105,User_106,User_107,User_108,User_109,User_11,User_110,User_111,User_112,User_113,User_114,User_115,User_116,User_117,User_118,User_119,User_12,User_120,User_121,User_122,User_123,User_124,User_125,User_126,User_127,User_128,User_129,User_13,User_130,User_131,User_132,User_133,User_134,User_135,User_136,User_137,User_138,User_139,User_14,User_140,User_141,User_142,User_143,User_144,User_145,User_146,User_147,User_148,User_149,User_15,User_16,User_17,User_18,User_19,User_2,User_20,User_21,User_22,User_23,User_24,User_25,User_26,User_27,User_28,User_29,User_3,User_30,User_31,User_32,User_33,User_34,User_35,User_36,User_37,User_38,User_39,User_4,User_40,User_41,User_42,User_43,User_44,User_45,User_46,User_47,User_48,User_49,User_5,User_50,User_51,User_52,User_53,User_54,User_55,User_56,User_57,User_58,User_59,User_6,User_60,User_61,User_62,User_63,User_64,User_65,User_66,User_67,User_68,User_69,User_7,User_70,User_71,User_72,User_73,User_74,User_75,User_76,User_77,User_78,User_79,User_8,User_80,User_81,User_82,User_83,User_84,User_85,User_86,User_87,User_88,User_89,User_9,User_90,User_91,User_92,User_93,User_94,User_95,User_96,User_97,User_98,User_99,W_Assess potential fraud__ate_abort,W_Assess potential fraud__complete,W_Assess potential fraud__resume,W_Assess potential fraud__schedule,W_Assess potential fraud__start,W_Assess potential fraud__suspend,W_Assess potential fraud__withdraw,W_Call after offers__ate_abort,W_Call after offers__complete,W_Call after offers__resume,W_Call after offers__schedule,W_Call after offers__start,W_Call after offers__suspend,W_Call after offers__withdraw,W_Call incomplete files__ate_abort,W_Call incomplete files__complete,W_Call incomplete files__resume,W_Call incomplete files__schedule,W_Call incomplete files__start,W_Call incomplete files__suspend,W_Complete application__ate_abort,W_Complete application__complete,W_Complete application__resume,W_Complete application__schedule,W_Complete application__start,W_Complete application__suspend,W_Handle leads__complete,W_Handle leads__resume,W_Handle leads__schedule,W_Handle leads__start,W_Handle leads__suspend,W_Handle leads__withdraw,W_Shortened completion __resume,W_Shortened completion __schedule,W_Shortened completion __start,W_Shortened completion __suspend,W_Validate application__ate_abort,W_Validate application__complete,W_Validate application__resume,W_Validate application__schedule,W_Validate application__start,W_Validate application__suspend
0,Created,User_112,W_Validate application__schedule,Workflow,Workitem_100000116,schedule,2016-03-02 13:25:52.388000+00:00,Home improvement,Limit raise,Application_1650545424,40000.0,0.0,0.0,0,0.0,0,0.0,0.0,0,Obtained,User_112,Workflow,Workitem_685532007,start,2016-03-02 13:25:52.390000+00:00,Home improvement,Limit raise,40000.0,0.0,0.0,0,0.0,0,0.0,0.0,0,0 days 00:00:00.002000,0.002,2.000000e+00,5.555556e-07,48352,2,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,6,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [3]:
N = 1000
n_processes = 12

import conduct_evaluation
get_pscores = lambda likelihoods : [pscore(likelihoods[1][i], likelihoods[2][k][3]).compute()[0] for i, k in enumerate(list(likelihoods[0].keys()))]

likelihoods_A = None
likelihoods_R = None
likelihoods_R_A = None
likelihoods_R_A_S = None
likelihoods_R_A_S_AC = None
likelihoods_R_A_S_RC = None
likelihoods_R_A_S_RC_AC = None
likelihoods_R_A_S_RC_AC_V = None
likelihoods_R_A_S_D = None
likelihoods_R_A_S_D_RC_AC = None

In [4]:
#needs strict_parser=False
#otherwise: KeyError: 'W_Assess potential fraud__ate_abort'
drbart_model_A = DRBART(parser_dir = '../../../models/'+model_name+'/concept-name/',
                     strict_parser=False)
evaluator_A = conduct_evaluation.ConductEvaluation(drbart_model_A, SampleOutcomes_DRBART_Normal_A, {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource_start',
                                                    },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_A = evaluator_A.sample_cases(False, True)

  0%|                                                                    | 0/6018 [00:00<?, ?it/s]

  0%|                                                         | 1/6018 [00:06<11:07:35,  6.66s/it]

  0%|                                                          | 4/6018 [00:07<2:30:25,  1.50s/it]

  0%|                                                          | 5/6018 [00:08<2:11:45,  1.31s/it]

  0%|                                                          | 6/6018 [00:10<2:45:47,  1.65s/it]

...

 31%|████████████████▉                                      | 1852/6018 [28:36<1:17:15,  1.11s/it]

 31%|████████████████▉                                      | 1853/6018 [28:37<1:13:55,  1.06s/it]

 31%|████████████████▉                                      | 1854/6018 [28:38<1:15:16,  1.08s/it]

 31%|████████████████▉                                      | 1854/6018 [28:38<1:04:19,  1.08it/s]

KeyError: 'W_Assess potential fraud__ate_abort'

In [5]:
np.mean([v.ln() for v in likelihoods_A[0].values()])

TypeError: 'NoneType' object is not subscriptable

In [6]:
np.mean(get_pscores(likelihoods_A))

TypeError: 'NoneType' object is not subscriptable

In [7]:
drbart_model_R_A = DRBART(parser_dir = '../../../models/'+model_name+'/concept-name_resource/',
                     strict_parser=False)
evaluator_R_A = conduct_evaluation.ConductEvaluation(drbart_model_R_A, SampleOutcomes_DRBART_Normal_R_A,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource_start',
                                                    },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_R_A = evaluator_R_A.sample_cases(False, True)

FileNotFoundError: [Errno 2] No such file or directory: '../../../models/bpic_2017_all/concept-name_resource/ucuts.json'

In [8]:
np.mean([v.ln() for v in likelihoods_R_A[0].values()])

TypeError: 'NoneType' object is not subscriptable

In [9]:
np.mean(get_pscores(likelihoods_R_A))

TypeError: 'NoneType' object is not subscriptable

In [10]:
drbart_model_R = DRBART(parser_dir = '../../../models/'+model_name+'/resource/',
                     strict_parser=False)
evaluator_R = conduct_evaluation.ConductEvaluation(drbart_model_R, SampleOutcomes_DRBART_Normal_R,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource_start',
                                                    },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_R = evaluator_R.sample_cases(False, True)

FileNotFoundError: [Errno 2] No such file or directory: '../../../models/bpic_2017_all/resource/ucuts.json'

In [11]:
np.mean([v.ln() for v in likelihoods_R[0].values()])

TypeError: 'NoneType' object is not subscriptable

In [12]:
np.mean(get_pscores(likelihoods_R))

TypeError: 'NoneType' object is not subscriptable

In [13]:
drbart_model_R_A_S = DRBART(parser_dir = '../../../models/'+model_name+'/concept-name_resource_seconds-in-day/',
                     strict_parser=False)
evaluator_R_A_S = conduct_evaluation.ConductEvaluation(drbart_model_R_A_S, SampleOutcomes_DRBART_Normal_R_A_S,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource_start',
                                                    },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_R_A_S = evaluator_R_A_S.sample_cases(False, True)

  0%|                                                                    | 0/6018 [00:00<?, ?it/s]

  0%|                                                         | 1/6018 [00:21<36:19:06, 21.73s/it]

  0%|                                                         | 2/6018 [00:29<22:36:20, 13.53s/it]

  0%|                                                         | 10/6018 [00:36<4:13:48,  2.53s/it]

  0%|                                                         | 12/6018 [00:37<3:22:35,  2.02s/it]

...

 99%|███████████████████████████████████████████████████████▎| 5945/6018 [00:27<00:00, 194.80it/s]

 99%|███████████████████████████████████████████████████████▌| 5966/6018 [00:27<00:00, 198.33it/s]

100%|███████████████████████████████████████████████████████▋| 5988/6018 [00:27<00:00, 198.53it/s]

100%|███████████████████████████████████████████████████████▉| 6010/6018 [00:27<00:00, 192.98it/s]

100%|████████████████████████████████████████████████████████| 6018/6018 [00:27<00:00, 217.90it/s]

In [14]:
np.mean([v.ln() for v in likelihoods_R_A_S[0].values()])

Decimal('-20.41065435346284260430650133')

In [15]:
np.mean(get_pscores(likelihoods_R_A_S))

np.float64(6.110400397145507e+21)

In [16]:
drbart_model_R_A_S_AC = DRBART(parser_dir = '../../../models/'+model_name+'/concept-name_resource_seconds-in-day_activity-count/',
                     strict_parser=False)
evaluator_R_A_S_AC = conduct_evaluation.ConductEvaluation(drbart_model_R_A_S_AC, SampleOutcomes_DRBART_Normal_R_A_S_AC,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource_start',
                                                        'known_activities' : known_activities,
                                                    },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_R_A_S_AC = evaluator_R_A_S_AC.sample_cases(False, True)

  0%|                                                                    | 0/6018 [00:00<?, ?it/s]

  0%|                                                         | 1/6018 [00:41<68:37:45, 41.06s/it]

  0%|                                                         | 2/6018 [01:17<64:22:52, 38.53s/it]

  0%|▎                                                        | 30/6018 [01:31<3:24:25,  2.05s/it]

  1%|▎                                                        | 37/6018 [01:41<3:08:06,  1.89s/it]

...

 99%|███████████████████████████████████████████████████████▍| 5954/6018 [00:33<00:00, 163.38it/s]

 99%|███████████████████████████████████████████████████████▌| 5972/6018 [00:33<00:00, 163.38it/s]

100%|███████████████████████████████████████████████████████▋| 5989/6018 [00:33<00:00, 164.01it/s]

100%|███████████████████████████████████████████████████████▉| 6008/6018 [00:33<00:00, 166.69it/s]

100%|████████████████████████████████████████████████████████| 6018/6018 [00:33<00:00, 177.13it/s]

In [17]:
np.mean([v.ln() for v in likelihoods_R_A_S_AC[0].values()])

Decimal('-9.740347812830777910185478764')

In [18]:
np.mean(get_pscores(likelihoods_R_A_S_AC))

np.float64(119884824999945.28)

In [19]:
drbart_model_R_A_S_RC = DRBART(parser_dir = '../../../models/'+model_name+'/concept-name_resource_seconds-in-day_resource-count/',
                     strict_parser=False)
evaluator_R_A_S_RC = conduct_evaluation.ConductEvaluation(drbart_model_R_A_S_RC, SampleOutcomes_DRBART_Normal_R_A_S_RC,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource_start',
                                                        'known_resources' : known_resources
                                                    },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_R_A_S_RC = evaluator_R_A_S_RC.sample_cases(False, True)

  0%|                                                                    | 0/6018 [00:00<?, ?it/s]

  0%|                                                         | 1/6018 [00:13<22:59:38, 13.76s/it]

  0%|                                                         | 2/6018 [00:42<37:24:47, 22.39s/it]

  0%|▏                                                        | 20/6018 [00:43<2:23:23,  1.43s/it]

  0%|▏                                                        | 23/6018 [00:46<2:21:02,  1.41s/it]

...

 99%|███████████████████████████████████████████████████████▍| 5955/6018 [00:36<00:00, 161.39it/s]

 99%|███████████████████████████████████████████████████████▌| 5972/6018 [00:36<00:00, 162.74it/s]

100%|███████████████████████████████████████████████████████▋| 5991/6018 [00:36<00:00, 169.31it/s]

100%|███████████████████████████████████████████████████████▉| 6011/6018 [00:36<00:00, 171.58it/s]

100%|████████████████████████████████████████████████████████| 6018/6018 [00:36<00:00, 164.09it/s]

In [20]:
np.mean([v.ln() for v in likelihoods_R_A_S_RC[0].values()])

Decimal('-5.434789166487137318274324393')

In [21]:
np.mean(get_pscores(likelihoods_R_A_S_RC))

np.float64(4289485.2108373325)

In [22]:
drbart_model_R_A_S_RC_AC = DRBART(parser_dir = '../../../models/'+model_name+'/concept-name_resource_seconds-in-day_activity-count_resource_count/',
                     strict_parser=False)
evaluator_R_A_S_RC_AC = conduct_evaluation.ConductEvaluation(drbart_model_R_A_S_RC_AC, SampleOutcomes_DRBART_Normal_R_A_S_RC_AC,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource_start',
                                                        'known_resources' : known_resources,
                                                        'known_activities' : known_activities,
                                                    },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_R_A_S_RC_AC = evaluator_R_A_S_RC_AC.sample_cases(False, True)

  0%|                                                                    | 0/6018 [00:00<?, ?it/s]

  0%|                                                         | 1/6018 [00:42<71:02:25, 42.50s/it]

  0%|                                                        | 2/6018 [02:04<109:59:38, 65.82s/it]

  0%|▎                                                        | 30/6018 [02:40<6:31:31,  3.92s/it]

  1%|▍                                                        | 52/6018 [03:02<3:59:13,  2.41s/it]

...

 99%|███████████████████████████████████████████████████████▎| 5938/6018 [00:31<00:00, 168.88it/s]

 99%|███████████████████████████████████████████████████████▍| 5958/6018 [00:31<00:00, 176.56it/s]

 99%|███████████████████████████████████████████████████████▋| 5978/6018 [00:31<00:00, 173.29it/s]

100%|███████████████████████████████████████████████████████▊| 6002/6018 [00:31<00:00, 186.11it/s]

100%|████████████████████████████████████████████████████████| 6018/6018 [00:31<00:00, 188.24it/s]

In [23]:
np.mean([v.ln() for v in likelihoods_R_A_S_RC_AC[0].values()])

Decimal('-Infinity')

In [24]:
np.mean(get_pscores(likelihoods_R_A_S_RC_AC))

np.float64(9.410392106551603e+16)

In [4]:
drbart_model_R_A_S_RC_AC_V = DRBART(parser_dir = '../../../models/'+model_name+'/concept-name_resource_seconds-in-day_activity-count_resource_count_amount/',
                     strict_parser=False)
evaluator_R_A_S_RC_AC_V = conduct_evaluation.ConductEvaluation(drbart_model_R_A_S_RC_AC_V, SampleOutcomes_DRBART_Normal_R_A_S_RC_AC_V,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource_start',
                                                        'value_key' : 'case:RequestedAmount_start',
                                                        'known_resources' : known_resources,
                                                        'known_activities' : known_activities,
                                                    },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_R_A_S_RC_AC_V = evaluator_R_A_S_RC_AC_V.sample_cases(False, False)

  1%|▌                                                             | 58/6018 [24:23<41:45:52, 25.23s/it]


KeyboardInterrupt: 

In [26]:
np.mean([v.ln() for v in likelihoods_R_A_S_RC_AC_V[0].values()])

TypeError: 'NoneType' object is not subscriptable

In [27]:
np.mean(get_pscores(likelihoods_R_A_S_RC_AC_V))

TypeError: 'NoneType' object is not subscriptable

In [28]:
drbart_model_R_A_S_D = DRBART(parser_dir = '../../../models/'+model_name+'/concept-name_resource_seconds-in-day_day-of-week/',
                     strict_parser=False)
evaluator_R_A_S_D = conduct_evaluation.ConductEvaluation(drbart_model_R_A_S_D, SampleOutcomes_DRBART_Normal_R_A_S_D,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource_start',
                                                    },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_R_A_S_D = evaluator_R_A_S_D.sample_cases(False, True)

  0%|                                                                    | 0/6018 [00:00<?, ?it/s]

  0%|                                                         | 1/6018 [00:31<52:28:22, 31.39s/it]

  0%|                                                         | 2/6018 [00:39<29:35:30, 17.71s/it]

  0%|                                                         | 10/6018 [00:41<4:08:37,  2.48s/it]

  0%|                                                         | 11/6018 [00:41<3:37:16,  2.17s/it]

...

 98%|███████████████████████████████████████████████████████▏| 5926/6018 [00:26<00:00, 230.89it/s]

 99%|███████████████████████████████████████████████████████▎| 5950/6018 [00:26<00:00, 225.99it/s]

 99%|███████████████████████████████████████████████████████▌| 5975/6018 [00:26<00:00, 228.62it/s]

100%|███████████████████████████████████████████████████████▊| 5999/6018 [00:27<00:00, 207.46it/s]

100%|████████████████████████████████████████████████████████| 6018/6018 [00:27<00:00, 221.88it/s]

In [29]:
np.mean([v.ln() for v in likelihoods_R_A_S_D[0].values()])

Decimal('-16.16958851133528924511241296')

In [30]:
np.mean(get_pscores(likelihoods_R_A_S_D))

np.float64(1.970780928662173e+19)

In [6]:
drbart_model_R_A_S_D_RC_AC = DRBART(parser_dir = '../../../models/'+model_name+'/concept-name_resource_seconds-in-day_day-of-week_activity-count_resoure-count/',
                     strict_parser=True)
evaluator_R_A_S_D_RC_AC = conduct_evaluation.ConductEvaluation(drbart_model_R_A_S_D_RC_AC, SampleOutcomes_DRBART_Normal_R_A_S_D_RC_AC,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource_start',
                                                        'known_resources' : known_resources,
                                                        'known_activities' : known_activities,
                                                    },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_R_A_S_D_RC_AC = evaluator_R_A_S_D_RC_AC.sample_cases(False, True)

  2%|█▏                                                            | 113/6018 [02:52<2:30:25,  1.53s/it]Process ForkPoolWorker-12:
Process ForkPoolWorker-10:
Process ForkPoolWorker-11:
Process ForkPoolWorker-1:
Process ForkPoolWorker-5:
Process ForkPoolWorker-9:
Process ForkPoolWorker-4:
Process ForkPoolWorker-8:
Process ForkPoolWorker-2:
Process ForkPoolWorker-6:
Process ForkPoolWorker-3:
Process ForkPoolWorker-7:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/LordKunkler/.pyenv/versions/3.10-dev/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/LordKunkler/.pyenv/versions/3.10-dev/lib/python3.10/multiprocessing/process.py", li

KeyboardInterrupt: 

In [32]:
np.mean([v.ln() for v in likelihoods_R_A_S_D_RC_AC[0].values()])

TypeError: 'NoneType' object is not subscriptable

In [33]:
np.mean(get_pscores(likelihoods_R_A_S_D_RC_AC))

TypeError: 'NoneType' object is not subscriptable

In [34]:
results = {
    'drbart_model_A' : likelihoods_A,
    'drbart_model_R' : likelihoods_R,
    'drbart_model_R_A' : likelihoods_R_A,
    'drbart_model_R_A_S' : likelihoods_R_A_S,
    'drbart_model_R_A_S_AC' : likelihoods_R_A_S_AC,
    'drbart_model_R_A_S_RC' : likelihoods_R_A_S_RC,
    'drbart_model_R_A_S_RC_AC' : likelihoods_R_A_S_RC_AC,
    'drbart_model_R_A_S_RC_AC_V' : likelihoods_R_A_S_RC_AC_V,
    'drbart_model_R_A_S_D' : likelihoods_R_A_S_D,
    'drbart_model_R_A_S_D_RC_CC' : likelihoods_R_A_S_D_RC_AC
}
with open('./'+model_name+'_dr_bart_evaluation_'+log_name+'.pickle', 'wb') as handle:
    pickle.dump(results, handle)